# 2-4 ネットワークモデルの実装

## SSD ネットワークモデルの概要
SSD のネットワークモデルの概要を次に示す．

<img src="../image/IMG_0194.jpg" alt="structure_ssd_network">

前処理を行った 300 x 300 の画像データを入力し，8,732個のデフォルトボックスに対するオフセット情報と21種類のクラスに対する信頼度を出力として得る．

vgg モジュールに入力された画像は VGG-16 モデルをベースにしたネットワークを通過する．
畳み込み層のカーネルサイズや使用するユニットは同じだが，各ユニットの出力する特徴量マップのサイズは VGG-16 とは異なる．
vgg モジュールで畳み込み層を10回通過したデータ（conv4_3）を抜き出し，L2 正規化を施して source1 とする．
この source1 はチャネル数512，38 x 38 の特徴量マップである．
また，vgg モジュールの最終的な出力を source2 とする．
source2 はチャネル数1,024，19 x 19 の特徴量マップである．

次に，source2 を extras モジュールに入力する．
ここでは，8回の畳み込みを行い2回ごとに特徴量マップを source3~6 として出力する．
それぞれの source のサイズは10x10，5x5，3x3，1x1となっている．
ここで重要になるのは，source の各特徴量マップがそれぞれ異なっていることである．
source1 では 38x38=1444 の領域について，それぞれの特徴量マップから物体を検出しようとしている一方で，source6　は画像全体を1つの大きな特徴量として捉え，大きな物体を検出しようとしている．
ただし，source によって畳み込みの回数が異なるため検出能力に差が出てくる．
例えば，source1 は10回の畳み込みしか通過していないのに対して，source6 は23回の畳み込みを受けている．
そのため，小さな領域の特徴量を十分に捉えきれず，SSD は小さな物体の検出には向かないという欠点がある．

こうして得られた source は loc モジュールと conf モジュールに入力され，それぞれからデフォルトボックスに対するオフセット情報と，各クラスに対する信頼度を出力する．
各 souce から得られる特徴量マップは合計で1,940個だが，source1, 5, 6 に対して4個，source2, 3, 4 に対しては6個のデフォルトボックスを用意している．
そのため，デフォルトボックスの合計は8,732個となる

## モジュール vgg を実装
vgg モジュールを実装する関数 make_vgg を作成する．
畳み込み層，ReLU 関数，マックスプーリングを合計で34ユニット使用する．
各畳み込み層のチャネル数とマックスプーリングの情報を cfg = $[64, 64, 'M', ・・・]$ として変数に格納しておく．
’M'はマックスプーリングを，’MC'はマックスプーリングを ceil モードで計算することを表している．
なお，ReLU の引数 inplace を True としておくと，ReLU への入力をメモリ上に保持せず上書きするため，メモリの節約になる．

## モジュール extras を実装
8個の畳み込みユニットを並べて extras モジュールを実装する make_extras 関数を実装する

## モジュール loc とモジュール conf を実装
loc と conf の各モジュールでは6つの畳み込み層が用意されているが，6つの畳み込み層を前から後ろへ順伝播するのではなく，それぞれの source に対応して1回ずつ計算される．
順伝播の実装は次節で定義する．

## L2Norm 層を実装
L2Norm 層ではチャネルごとに特徴量マップが異なる点を正規化する．
ここでは，1,444個の特徴量マップのセルごとに2乗和を計算し，その平方根で割り算して正規化を行う．
さらに，512x38x38のテンソルに対して係数をかける．
この512個の係数は学習パラメータである．

## デフォルトボックスを実装
それぞれの source に対して4個または6個のデフォルトボックスを作成する．
デフォルトボックスが4つの場合，小さい正方形，大きい正方形，1:2の長方形，2:1の長方形で，6種類の場合にはこれに加えて3:1と1:3の長方形を用意する．

In [ ]:
# デフォルトボックスの実装

In [1]:
# 動作確認

## クラス SSD を実装する
最後に，PyTorch のネットワーク層クラス nn.Module を継承し SSD クラスを実装する．
ただし，順伝播のメソッドは次節で追加する．